<a href="https://colab.research.google.com/github/dushyant4342/Fake_News_Classifier/blob/master/Fake_News_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_fakenews.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
X=df.drop('label', axis=1)
y=df['label']

In [ ]:
X.shape, y.shape

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][0:5]

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
df.head(10)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

In [ ]:
len(messages)

In [ ]:
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
len(corpus)

In [ ]:
corpus

In [ ]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

In [ ]:
sent_length=20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
embedded_docs.shape

In [ ]:
embedded_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedded_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
model.summary()

In [ ]:
len(embedded_docs), y.shape

In [ ]:
import numpy as np

In [ ]:
X_final=np.array(embedded_docs)
X_final

In [ ]:
y_final=np.array(y)

In [ ]:
X_final.shape, y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test =train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [ ]:
X_train.shape

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_test, y_pred)